In [22]:
import re
import random

data_path = "model/human_text.txt"
data_path2 = "model/robot_text.txt"
# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
with open(data_path2, 'r', encoding='utf-8') as f:
  lines2 = f.read().split('\n')
lines = [re.sub(r"\[\w+\]",'hola',line) for line in lines]
lines = [" ".join(re.findall(r"\w+",line.lower())) for line in lines]
lines2 = [re.sub(r"\[\w+\]",'',line) for line in lines2]
lines2 = [" ".join(re.findall(r"\w+",line.lower())) for line in lines2]
# Grouping lines by response pair
pairs = list(zip(lines,lines2))
#random.shuffle(pairs)

In [23]:
import numpy as np

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs:
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [24]:
print(pairs[:5])
print(input_docs[:5])

[('hola buenos días', 'buenos días en que puedo servirle'), ('tienes domicilio', 'claro que si que deseas pedir a domicilio'), ('que cuesta la arepa de huevo', 'la arepa tiene un costo de 2000 pesos sin incluir el domicilio'), ('y vendes chocolatico caliente', 'claro el chocolate un costo de 1000 sin leche'), ('y con leche', 'con lehe tiene un costo de 2000 pesos')]
['hola buenos días', 'tienes domicilio', 'que cuesta la arepa de huevo', 'y vendes chocolatico caliente', 'y con leche']


In [31]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 300
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='ADAM', loss='categorical_crossentropy')
#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('model/training_model.h5')

Epoch 1/300
21/21 [==============================] - 11s 352ms/step - loss: 1.5044 - val_loss: 1.3159
Epoch 2/300
21/21 [==============================] - 6s 307ms/step - loss: 1.3524 - val_loss: 1.2727
Epoch 3/300
21/21 [==============================] - 6s 307ms/step - loss: 1.1940 - val_loss: 1.2834
Epoch 4/300
21/21 [==============================] - 6s 305ms/step - loss: 1.2387 - val_loss: 1.2728
Epoch 5/300
21/21 [==============================] - 6s 308ms/step - loss: 1.2729 - val_loss: 1.2705
Epoch 6/300
21/21 [==============================] - 6s 305ms/step - loss: 1.3192 - val_loss: 1.2697
Epoch 7/300
21/21 [==============================] - 6s 305ms/step - loss: 1.1812 - val_loss: 1.2660
Epoch 8/300
21/21 [==============================] - 6s 308ms/step - loss: 1.3429 - val_loss: 1.2726
Epoch 9/300
21/21 [==============================] - 6s 308ms/step - loss: 1.2916 - val_loss: 1.2716
Epoch 10/300
21/21 [==============================] - 6s 308ms/step - loss: 1.2214 - val_l

21/21 [==============================] - 6s 306ms/step - loss: 0.8483 - val_loss: 1.1308
Epoch 82/300
21/21 [==============================] - 6s 304ms/step - loss: 0.9024 - val_loss: 1.1295
Epoch 83/300
21/21 [==============================] - 6s 304ms/step - loss: 0.7607 - val_loss: 1.1278
Epoch 84/300
21/21 [==============================] - 6s 306ms/step - loss: 0.7465 - val_loss: 1.1282
Epoch 85/300
21/21 [==============================] - 6s 306ms/step - loss: 0.9095 - val_loss: 1.1275
Epoch 86/300
21/21 [==============================] - 6s 305ms/step - loss: 0.8437 - val_loss: 1.1235
Epoch 87/300
21/21 [==============================] - 6s 305ms/step - loss: 0.8440 - val_loss: 1.1245
Epoch 88/300
21/21 [==============================] - 6s 305ms/step - loss: 0.8020 - val_loss: 1.1240
Epoch 89/300
21/21 [==============================] - 6s 305ms/step - loss: 0.8732 - val_loss: 1.1251
Epoch 90/300
21/21 [==============================] - 6s 304ms/step - loss: 0.7504 - val_loss: 

Epoch 161/300
21/21 [==============================] - 6s 305ms/step - loss: 0.5648 - val_loss: 1.0870
Epoch 162/300
21/21 [==============================] - 6s 306ms/step - loss: 0.5478 - val_loss: 1.0763
Epoch 163/300
21/21 [==============================] - 6s 304ms/step - loss: 0.5178 - val_loss: 1.0723
Epoch 164/300
21/21 [==============================] - 6s 305ms/step - loss: 0.5820 - val_loss: 1.0614
Epoch 165/300
21/21 [==============================] - 6s 306ms/step - loss: 0.5176 - val_loss: 1.0675
Epoch 166/300
21/21 [==============================] - 6s 305ms/step - loss: 0.5280 - val_loss: 1.0673
Epoch 167/300
21/21 [==============================] - 7s 315ms/step - loss: 0.5247 - val_loss: 1.0635
Epoch 168/300
21/21 [==============================] - 6s 306ms/step - loss: 0.5184 - val_loss: 1.0668
Epoch 169/300
21/21 [==============================] - 6s 305ms/step - loss: 0.5246 - val_loss: 1.0599
Epoch 170/300
21/21 [==============================] - 6s 307ms/step - lo

21/21 [==============================] - 9s 426ms/step - loss: 0.3864 - val_loss: 1.0848
Epoch 241/300
21/21 [==============================] - 7s 314ms/step - loss: 0.3577 - val_loss: 1.0870
Epoch 242/300
21/21 [==============================] - 7s 317ms/step - loss: 0.3711 - val_loss: 1.0807
Epoch 243/300
21/21 [==============================] - 6s 311ms/step - loss: 0.3668 - val_loss: 1.0841
Epoch 244/300
21/21 [==============================] - 6s 308ms/step - loss: 0.3371 - val_loss: 1.0881
Epoch 245/300
21/21 [==============================] - 6s 307ms/step - loss: 0.3924 - val_loss: 1.0836
Epoch 246/300
21/21 [==============================] - 6s 306ms/step - loss: 0.3713 - val_loss: 1.0865
Epoch 247/300
21/21 [==============================] - 6s 310ms/step - loss: 0.3303 - val_loss: 1.0815
Epoch 248/300
21/21 [==============================] - 6s 310ms/step - loss: 0.3365 - val_loss: 1.0852
Epoch 249/300
21/21 [==============================] - 6s 305ms/step - loss: 0.3403 - v

In [26]:
training_model.save('model/training_model2.h5')

In [32]:
from keras.models import load_model
training_model = load_model('model/training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    #A variable to store our response word by word
    decoded_sentence = ''
    
    stop_condition = False
    while not stop_condition:
          #Predicting output tokens with probabilities and states
          output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
    #Choosing the one with highest probability
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_token = reverse_target_features_dict[sampled_token_index]
          decoded_sentence += " " + sampled_token
    #Stop if hit max length or found the stop token
          if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
    #Update the target sequence
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.
          #Update states
          states_value = [hidden_state, cell_state]
    return decoded_sentence

In [ ]:
class ChatBot:
  negative_responses = ("no", "negativo", "nada", "nop", "nopi")
  exit_commands = ("chao", "adios", "nos vemos",
                 "suerte", "chaito", "bye", "chaolin")
#Method to start the conversation
  def start_chat(self):
    user_response = input("Hola, bienvenido en que te puedo ayudar?\n")
    
    if user_response in self.negative_responses:
      print("Ok, perfecto, que tengas un hermoso día!")
      return
    self.chat(user_response)
#Method to handle the conversation
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Remove <START> and <END> tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response
#Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Que tengas un hermoso día!")
        return True
    return False
  
chatbot = ChatBot()
chatbot.start_chat()

Hola, bienvenido en que te puedo ayudar?
Hola, buen día
 buen día en que puedo 
Quiero una pizza pequeña por favor
 claro que si nuestro horario de atención
si
 listo es con mucho gusto gracias por preferirnos
feliz dia
 el como aplica para la 
ok
 nuestro horario de atención es martes
pruebas
 si productos postobon y cocacola 
cocacola
 esta bién deseas que te envie algo más
que cuesta la arepa de huevo?
 la arepa tiene un costo de 2000 pesos
y vendes chocolatico caliente?
 claro el chocolate un costo de 1000 sin
Gaseosa manejas?


In [29]:
chatbot.start_chat()

Hola, bienvenido en que te puedo ayudar?
Hola,buenas tardes
 es ensalada de 
hol
 no entiendo lo que me dices esto 
hola
 hola ya te envio nuestro 
Hola, buenos dias
 buenos días en que te puedo servirle 
hola, buenos días
 buenos días en que puedo servirle 
hola, buenas tardes
 hola como estas 
tienes pizza?
 a 10000 el trifásic y a 8000 el de 
tienes pizza grande
 a 10000 el trifásic y a 8000 el de 
hacen domicilio?
 claro que si cuántos maicitos desea 
cuánto cuesta el domicilio?
 así es tenemos varias opciones vegetarianas
tienen sánduches?
 muy bien en que puedo servirle 
tiene sanduches?
 estamos ubicados en adiciones 
adios
Que tengas un hermoso día!
